In [1]:
import numpy as np
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import io
import os
import random as rd
import csv
from scipy.signal import convolve2d, convolve


C:\Users\dylan\AppData\Local\Temp\ipykernel_13928\2904952402.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Fonction utilitaires :

def export_list_to_csv(data, file_name):
    csv_file_path = file_name + ".csv"
    with open(csv_file_path, 'w', newline='') as csvfile:
        # Create a CSV writer object
        csv_writer = csv.writer(csvfile)
        
        # Write each sublist as a row in the CSV file
        for sublist in data:
            csv_writer.writerow(sublist)


def csv_to_list(csv_file_path):
    data_list = []
    with open(csv_file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        
        for row in csv_reader:
            row_list = [[int(num) for num in string.strip('[]').split()] for string in row]
            data_list.append(row_list)
    return data_list


In [3]:
# Test de détection d'outliers

def outlier_final(array):
    length = array.shape[0]
    L = []
    kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
    
    # Accumulation sur 4 heures 
    data_4_hours = np.array([np.sum(array[48 * i:48 * (i + 1)], axis=0) for i in range(length // 48)])
    

    S = np.array([convolve2d(element, kernel, mode='same') for element in data_4_hours])
    
    for t in range(len(S)):
        St = S[t]
        Dt = data_4_hours[t]
        frontier_Dt = np.mean(Dt) + 5 * np.std(Dt)
        frontier_St = np.mean(St) + 30 * np.std(St)
        
        A = np.argwhere((np.abs(St) > frontier_St) & (np.abs(Dt) > frontier_Dt))
        
        for _ in range(48):
            L.append(A)
            
    return L

In [4]:
zip_filename = 'outliers_per_days.zip'

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    pass


In [5]:
zip_file_path = "data/2018.zip"

with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        all_files = zip_ref.namelist()
        for file in all_files:
            if file.endswith(".npy"):
                year_month_day = file[19:27]
                print(year_month_day)
                with zip_ref.open(file) as file_in_zip:
                    npy_bytes = file_in_zip.read()
                    npy_file_like = io.BytesIO(npy_bytes)
                    npy_array = np.load(npy_file_like)
                    outlier_list = outlier_final(npy_array)
                    csv_data = io.StringIO()
                    writer = csv.writer(csv_data)
                    for row in outlier_list:
                        writer.writerow(row)

                    # Nom du fichier CSV dans le ZIP
                    csv_filename = f'outliers_{year_month_day}.csv'

                    # Nom du fichier ZIP existant
                    zip_filename = 'outliers_per_days.zip'

                    # Ajouter le fichier CSV au fichier ZIP
                    with zipfile.ZipFile(zip_filename, 'a') as zipf:
                        zipf.writestr(csv_filename, csv_data.getvalue())

                    print(f'Fichier {csv_filename} ajouté à {zip_filename}')
                
                    npy_file_like.close()


20180324
Fichier outliers_20180324.csv ajouté à outliers_per_days.zip
20180325
Fichier outliers_20180325.csv ajouté à outliers_per_days.zip
20180326
Fichier outliers_20180326.csv ajouté à outliers_per_days.zip
20180327
Fichier outliers_20180327.csv ajouté à outliers_per_days.zip
20180328
Fichier outliers_20180328.csv ajouté à outliers_per_days.zip
20180329
Fichier outliers_20180329.csv ajouté à outliers_per_days.zip
20180330
Fichier outliers_20180330.csv ajouté à outliers_per_days.zip
20180331
Fichier outliers_20180331.csv ajouté à outliers_per_days.zip
20180401
Fichier outliers_20180401.csv ajouté à outliers_per_days.zip
20180402
Fichier outliers_20180402.csv ajouté à outliers_per_days.zip
20180403
Fichier outliers_20180403.csv ajouté à outliers_per_days.zip
20180404
Fichier outliers_20180404.csv ajouté à outliers_per_days.zip
20180405
Fichier outliers_20180405.csv ajouté à outliers_per_days.zip
20180406
Fichier outliers_20180406.csv ajouté à outliers_per_days.zip
20180407
Fichier out